# Multiband Images

In [1]:
##################################################
#
# Libraries
#





from metpy.plots    import colortables
from metpy.plots    import add_timestamp
from datetime       import datetime
from siphon.catalog import TDSCatalog
from datetime       import datetime


import numpy             as np
import os                as os
import xarray            as xr
import pandas            as pd
import pathlib           as pathlib

import metpy
import cartopy.crs       as ccrs
import cartopy.feature   as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as patches





#
##################################################

In [2]:
####################################################
####################################################
####################################################
#
# Mines Colors and Fonts
#

Mines_Blue = "#002554"


plt.rcParams.update({'text.color'      : Mines_Blue,
                     'axes.labelcolor' : Mines_Blue,
					 'axes.edgecolor'  : Mines_Blue,
					 'xtick.color'     : Mines_Blue,
					 'ytick.color'     : Mines_Blue})


#
####################################################
####################################################
####################################################

## Get Time Step here.

https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/catalog.html



In [3]:


YYYYMMDD         = "20230518"

YYYDOYhhmmssf    = "20231382356170"

file_time_string = "s" + YYYDOYhhmmssf + "_e" + YYYDOYhhmmssf + "_c" + YYYDOYhhmmssf


In [4]:
##################################################
#
# Channel Labels
#

MAINDIR = os.getcwd() +"/"
print(MAINDIR)

alpha_factor = 0.05



channel_lab = [' Channel Zero',                     #  0
               ' [0.47 µm Blue-Visible]',           #  1
               ' [0.64 µm Red-Visible]',            #  2
               ' [0.86 µm Vegetation Near-IR]',     #  3
               ' [1.37 µm Cirrus Near-IR]',         #  4
               ' [1.6 µm Snow/Ice Near-IR]',        #  5
               ' [2.2 µm Cloud Particle Near-IR]',  #  6
               ' [3.9 µm Middle Infrared]',         #  7
               ' [6.2 µm Upper-Level Water Vapor]', #  8
               ' [6.9 µm Mid-Level Water Vapor]',   #  9
               ' [7.3 µm Low-Level Water Vapor]',   # 10
               ' [8.4 µm Cloud-Top Infrared]',      # 11
               ' [9.6 µm Ozone Infrared]',          # 12
               ' [10.3 µm Clean IR Window]',        # 13
               ' [11.2 µm Middle IR Window]',       # 14
               ' [12.3 µm Dirty IR Window]',        # 15
               ' [13.3 µm CO₂ Infrared]']           # 16



png_file_dir3  = "./temp_files_sat_sodak/"



imin_rap_tir =  551 # i_rap-250
imax_rap_tir = 1051 # i_rap+250
jmin_rap_tir =    0 # j_rap-250+19
jmax_rap_tir =  500 # j_rap+250-19

imin_rap_vis = 1103*2 # np.argmin(np.abs(x_vis-x_min_t).values)
imax_rap_vis = 2102*2 # np.argmin(np.abs(x_vis-x_max_t).values)
jmin_rap_vis =    0*2 # np.argmin(np.abs(y_vis-y_min_t).values)
jmax_rap_vis = 1000*2 # np.argmin(np.abs(y_vis-y_max_t).values)

vis_ny =  6000
vis_nx = 10000


imin_rap_frac = imin_rap_vis / vis_nx
imax_rap_frac = imax_rap_vis / vis_nx

jmin_rap_frac = jmin_rap_vis / vis_ny
jmax_rap_frac = jmax_rap_vis / vis_ny

print(imin_rap_frac,imax_rap_frac)
print(jmin_rap_frac,jmax_rap_frac)

#
##################################################

/Users/wjc/GitHub/SD_Mines_Map_Wall/GOES_Imagery/
0.2206 0.4204
0.0 0.3333333333333333


# SODAK Subset form 

In [5]:




##################################################

In [18]:
##################################################
#
# Control Setup
#

# %load solutions/data_url.py


# Cell content replaced by load magic replacement.

# Create variables for URL generation

for ch_band in range(1,17):

    channel        = ch_band
    ch_band_string = str(ch_band).zfill(2)


    image_date  = datetime.utcnow().date()
    region      =                   'CONUS'



    if (region == 'CONUS') :
        region_lab               = ' SODAK Band '
        png_processing_directory = png_file_dir3
        gif_file_name            = "./SD_BAND_" + ch_band_string + "_"





    # We want to match something like:
    # https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

    # Construct the data_url string


    data_url = "https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel"+ \
                ch_band_string+"/"+YYYYMMDD+"/OR_ABI-L2-CMIPC-M6C"+ch_band_string+"_G16_"+file_time_string+".nc"

    # Print out your URL and verify it works!

    print(data_url)


    ds  = xr.open_dataset(data_url)


    nx = ds.dims["x"]
    ny = ds.dims["y"]

    imin = round(imin_rap_frac * nx)
    imax = round(imax_rap_frac * nx)

    jmin = round(jmin_rap_frac * ny)
    jmax = round(jmax_rap_frac * ny)


    dat = ds.metpy.parse_cf('Sectorized_CMI')[jmin : jmax,
                                              imin : imax]



    x    = dat['x']       
    y    = dat['y']
    proj = dat.metpy.cartopy_crs

    tz         = 'America/Denver'
    time_utc   = datetime.strptime(ds.start_date_time, '%Y%j%H%M%S')
    valid_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d %H:%M:%S %Z")
    local_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H:%M:%S %Z")

    file_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d_%H%M")

    print(valid_time,local_time, file_time)

    image_header_label = "GOES 16" + region_lab + str(channel)+ channel_lab[channel]



    ny = dat.shape[0]
    nx = dat.shape[1]      
    alpha2d = np.sqrt(np.outer(np.abs(np.hanning(ny)),np.abs(np.hanning(nx))))
    alpha2d = np.where(alpha2d>alpha_factor,alpha_factor,alpha2d)
    alpha2d = alpha2d / alpha_factor




    fig = plt.figure(figsize=(8, 8), facecolor = 'white')

    plt.suptitle(image_header_label,
                 fontsize = 20, 
                 color    = Mines_Blue)
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    ax.set_title(valid_time + "  (" + local_time+")",
                 fontsize =      15, 
                 color    = Mines_Blue)
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=1, edgecolor=Mines_Blue)
    ax.add_feature(cfeature.STATES.with_scale('50m'),    linestyle=':', edgecolor=Mines_Blue)
    ax.add_feature(cfeature.BORDERS.with_scale('50m'),   linewidth=1, edgecolor=Mines_Blue)

    print(ch_band_string, dat.min().values, dat.max().values)

    
    vmin =  np.nanpercentile(a=dat.values, q=2)
    vmax =  np.nanpercentile(a=dat.values, q=98)
    print(ch_band_string, vmin, vmax)

    if (ch_band >= 7):
        colortab  = "gist_ncar"
        bar_label = "Top-of-Atmosphere Radiometric Temperature (K)"
        vmin = 205
        vmax = 295

    else:
        colortab  = "Greys_r"
        bar_label = "Top-of-Atmosphere Reflectance (fraction)"
        vmin = 0.
        vmax = 0.3
        
    im = ax.imshow(                       dat, 
                   extent = (x.min(), x.max(), 
                             y.min(), y.max()), 
                   origin =            'upper',
                   cmap   =          colortab,alpha = alpha2d,
                   vmin   =              vmin,
                   vmax   =              vmax)


    plt.colorbar(im,
                ax=ax,
                orientation = "horizontal",
                pad = 0,
                label = bar_label)
    
    


    #########################################
    #
    # Insert a Clock
    #

    axins = fig.add_axes(rect     =    [0.002,
                                        0.825,
                                        0.12,
                                        0.12],
                          projection  =  "polar")

    time_for_clock = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).time()

    hour   = time_for_clock.hour
    minute = time_for_clock.minute
    second = time_for_clock.second

    circle_theta  = np.deg2rad(np.arange(0,360,0.01))
    circle_radius = circle_theta * 0 + 1

    if (hour > 12) :
        hour = hour - 12

    angles_h = 2*np.pi*hour/12+2*np.pi*minute/(12*60)+2*second/(12*60*60)
    angles_m = 2*np.pi*minute/60+2*np.pi*second/(60*60)

    #print(time_for_clock)
    #print(hour,   np.rad2deg(angles_h))
    #print(minute, np.rad2deg(angles_m))


    plt.setp(axins.get_yticklabels(), visible=False)
    plt.setp(axins.get_xticklabels(), visible=False)
    axins.spines['polar'].set_visible(False)
    axins.set_ylim(0,1)
    axins.set_theta_zero_location('N')
    axins.set_theta_direction(-1)
    axins.set_facecolor("white")
    axins.grid(False)

    axins.plot([angles_h,angles_h], [0,0.60], color=Mines_Blue, linewidth=1.5)
    axins.plot([angles_m,angles_m], [0,0.95], color=Mines_Blue, linewidth=1.5)
    axins.plot(circle_theta, circle_radius, color=Mines_Blue, linewidth=1)

    #
    #########################################



    #########################################
    #
    # Insert a Footprint Map
    #   


    axmap = fig.add_axes(rect        =    [0.88, 
                                           0.825,
                                           0.12, 
                                           0.12],
                         projection = proj)

    axmap.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor=Mines_Blue)

    footprint_xy=np.array([[x.min(),y.min()],
                           [x.min(),y.max()],
                           [x.max(),y.max()],
                           [x.max(),y.min()],
                           [x.min(),y.min()]])

    footprint = patches.Polygon(xy        = footprint_xy,
                                facecolor =        'c')


    axmap.add_patch(footprint)

    axmap.set_global()

    #
    #########################################

    ax.set_frame_on(False)
    plt.tight_layout()
    dataset_png_file_name=png_file_dir3 +  "./SODAK_CH" + ch_band_string + "_" + file_time
    plt.savefig( dataset_png_file_name,
                    facecolor   = 'white', 
                    transparent =   False)
    plt.close()



#
##################################################

https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel01/20230518/OR_ABI-L2-CMIPC-M6C01_G16_s20231382356170_e20231382356170_c20231382356170.nc
2023-05-18 23:56:17 UTC 2023-05-18 17:56:17 MDT 2023-05-18_2356
01 0.037460282 0.74539614
01 0.0711110457777977 0.4336503744125366


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_32610/3763670139.py:222: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel02/20230518/OR_ABI-L2-CMIPC-M6C02_G16_s20231382356170_e20231382356170_c20231382356170.nc
2023-05-18 23:56:17 UTC 2023-05-18 17:56:17 MDT 2023-05-18_2356
02 0.01174602 0.7295231
02 0.03523806110024452 0.39460280537605286


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_32610/3763670139.py:222: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel03/20230518/OR_ABI-L2-CMIPC-M6C03_G16_s20231382356170_e20231382356170_c20231382356170.nc
2023-05-18 23:56:17 UTC 2023-05-18 17:56:17 MDT 2023-05-18_2356
03 0.01142856 0.81269765
03 0.07365072518587112 0.45460274815559387


...

/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_32610/3763670139.py:222: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel15/20230518/OR_ABI-L2-CMIPC-M6C15_G16_s20231382356170_e20231382356170_c20231382356170.nc
2023-05-18 23:56:17 UTC 2023-05-18 17:56:17 MDT 2023-05-18_2356
15 204.35123 294.88367
15 213.52359008789062 288.9871520996094


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_32610/3763670139.py:222: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel16/20230518/OR_ABI-L2-CMIPC-M6C16_G16_s20231382356170_e20231382356170_c20231382356170.nc
2023-05-18 23:56:17 UTC 2023-05-18 17:56:17 MDT 2023-05-18_2356
16 206.38828 272.43103
16 214.37509155273438 270.33795166015625


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_32610/3763670139.py:222: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


## Aggregate All Three Into One